# Labeling
# 제거할 코인 리스트
coins_to_remove = ['WLFI-USD', 'AETH-USD']
# 추가할 코인 리스트
coins_to_add = ['ALGO-USD', 'SEI-USD']

In [16]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

class CryptoLabeler:
    def __init__(self):
        """A00_00에서 생성된 데이터에 맞는 라벨링 클래스"""
        self.future_periods = [1, 3, 7, 14, 30, 90]
        
    def create_composite_score(self, df):
        """A00_00 데이터 구조에 맞는 복합 점수 생성"""
        try:
            if not isinstance(df, pd.DataFrame) or df.empty:
                return pd.Series(0.0, index=range(1)), {}
            
            # A00_00에서 이미 계산된 Future_Return 컬럼들 사용
            available_returns = {}
            
            # 가중치 설정 (단기에 더 큰 가중치)
            weights = {
                'Future_Return_1d': 0.30,
                'Future_Return_3d': 0.30, 
                'Future_Return_7d': 0.20,
                'Future_Return_14d': 0.10,
                'Future_Return_30d': 0.07,
                'Future_Return_90d': 0.03
            }
            
            # 실제 존재하는 Future_Return 컬럼들 확인
            for col in df.columns:
                if col.startswith('Future_Return_') and col in weights:
                    future_return = pd.to_numeric(df[col], errors='coerce')
                    if not future_return.isna().all():
                        available_returns[col] = future_return
            
            if not available_returns:
                print("⚠️ Future_Return 컬럼을 찾을 수 없습니다.")
                return pd.Series(0.0, index=df.index), {}
            
            # 실제 존재하는 컬럼들의 가중치 정규화
            total_weight = sum(weights.get(key, 0) for key in available_returns.keys())
            if total_weight > 0:
                normalized_weights = {key: weights.get(key, 0)/total_weight for key in available_returns.keys()}
            else:
                normalized_weights = {}
            
            # 수익률 기반 점수 계산
            return_score = pd.Series(0.0, index=df.index)
            
            for key, future_return in available_returns.items():
                if key in normalized_weights and normalized_weights[key] > 0:
                    clean_return = future_return.fillna(0)
                    return_score += normalized_weights[key] * clean_return
            
            # 최종 복합 점수 (-2.5 ~ +2.5 범위로 스케일링)
            composite_score = np.clip(return_score, -0.25, 0.25) * 10
            
            return composite_score, available_returns
        
        except Exception as e:
            print(f"⚠️ create_composite_score 오류: {e}")
            return pd.Series(0.0, index=df.index if isinstance(df, pd.DataFrame) else range(1)), {}

    def score_to_label(self, composite_score):
        """점수를 3단계 라벨로 변환 (0: 강매도, 1: 홀드, 2: 강매수)"""
        try:
            # 유효한 점수만 추출 (NaN 제외)
            valid_scores = composite_score.dropna()
            
            if len(valid_scores) == 0:
                return pd.Series([1] * len(composite_score), index=composite_score.index)
            
            # 백분위수 기준으로 라벨 생성
            labels = pd.Series(1, index=composite_score.index)  # 기본값: 홀드
            
            # 동적 임계값 설정 (데이터 분포에 따라 조정)
            if len(valid_scores) >= 50:
                # 충분한 데이터가 있을 때: 3분할
                buy_threshold = np.percentile(valid_scores, 55)   # 상위 45%
                sell_threshold = np.percentile(valid_scores, 45)  # 하위 45%
            else:
                # 적은 데이터일 때: 더 보수적인 기준
                buy_threshold = np.percentile(valid_scores, 55)   # 상위 
                sell_threshold = np.percentile(valid_scores, 45)  # 하위 
            
            labels[composite_score >= buy_threshold] = 2  # 강매수
            labels[composite_score <= sell_threshold] = 0  # 강매도
            
            return labels
        
        except Exception as e:
            print(f"⚠️ score_to_label 오류: {e}")
            return pd.Series([1] * len(composite_score), index=composite_score.index)

    def calculate_technical_score(self, df):
        """A00_00 데이터의 기술적 지표를 활용한 기술적 점수 계산"""
        try:
            score = pd.Series(0.0, index=df.index)
            weight_sum = 0
            
            # RSI 관련 점수 (과매수/과매도 상태 활용)
            if 'RSI_14_overbought' in df.columns and 'RSI_14_oversold' in df.columns:
                rsi_score = pd.Series(0.0, index=df.index)
                rsi_score[df['RSI_14_oversold'] == 1] = 1.0  # 과매도 → 매수 신호
                rsi_score[df['RSI_14_overbought'] == 1] = -1.0  # 과매수 → 매도 신호
                score += rsi_score * 0.2
                weight_sum += 0.2
            
            # Stochastic 관련 점수
            stoch_cols = [col for col in df.columns if 'Stoch_' in col and '_K_above_D' in col]
            if stoch_cols:
                stoch_score = pd.Series(0.0, index=df.index)
                for col in stoch_cols[:3]:  # 상위 3개만 사용
                    stoch_score += (df[col] * 2 - 1) * 0.1  # 0,1 → -1,1 변환
                score += stoch_score
                weight_sum += 0.3
            
            # CCI 관련 점수
            if 'CCI_3_overbought' in df.columns and 'CCI_3_oversold' in df.columns:
                cci_score = pd.Series(0.0, index=df.index)
                cci_score[df['CCI_3_oversold'] == 1] = 1.0  # 과매도
                cci_score[df['CCI_3_overbought'] == 1] = -1.0  # 과매수
                score += cci_score * 0.15
                weight_sum += 0.15
            
            # SMI 관련 점수
            if 'SMI_normalized' in df.columns:
                smi_score = pd.to_numeric(df['SMI_normalized'], errors='coerce').fillna(0)
                score += smi_score * 0.2
                weight_sum += 0.2
            
            # 모멘텀 관련 점수
            if 'Momentum_Signal' in df.columns:
                momentum_score = (df['Momentum_Signal'] - 1)  # 0,1,2 → -1,0,1
                score += momentum_score * 0.15
                weight_sum += 0.15
            
            # 가중치 정규화
            if weight_sum > 0:
                score = score / weight_sum
            
            return score
            
        except Exception as e:
            print(f"⚠️ calculate_technical_score 오류: {e}")
            return pd.Series(0.0, index=df.index)

    def create_enhanced_labels(self, df):
        """복합적인 라벨 생성 (기술적 점수 + 미래 수익률)"""
        try:
            # 1. 기술적 점수 계산
            technical_score = self.calculate_technical_score(df)
            
            # 2. 복합 점수 생성 (미래 수익률 기반)
            composite_score, future_returns_dict = self.create_composite_score(df)
            
            # 3. 가중 평균으로 최종 점수 계산
            # final_score = 0.4 * technical_score + 0.6 * composite_score
            final_score = composite_score
            # 4. 라벨 생성
            labels = self.score_to_label(final_score)
            
            # 5. 라벨 정보 추가
            label_names = {0: 'Strong_Sell', 1: 'Hold', 2: 'Strong_Buy'}
            label_name_series = labels.map(label_names)
            
            return {
                'Label': labels,
                'Label_Name': label_name_series,
                'Technical_Score': technical_score,
                'Composite_Score': composite_score,
                'Final_Score': final_score,
                'Future_Returns': future_returns_dict
            }
            
        except Exception as e:
            print(f"⚠️ create_enhanced_labels 오류: {e}")
            return None

    def process_crypto_csv(self, file_path, symbol_name):
        """개별 CSV 파일 처리"""
        try:
            # CSV 파일 읽기
            df = pd.read_csv(file_path)
            
            if df.empty:
                print(f"❌ {symbol_name}: 빈 데이터")
                return None
            
            # Date 컬럼을 datetime으로 변환
            if 'Date' in df.columns:
                df['Date'] = pd.to_datetime(df['Date'])
                df = df.sort_values('Date').reset_index(drop=True)
            
            # 최소 데이터 길이 확인
            if len(df) < 30:
                print(f"❌ {symbol_name}: 데이터 부족 ({len(df)}일)")
                return None
            
            # 라벨 생성
            label_results = self.create_enhanced_labels(df)
            
            if label_results is None:
                print(f"❌ {symbol_name}: 라벨 생성 실패")
                return None
            
            # 결과를 DataFrame에 추가
            for key, value in label_results.items():
                if key != 'Future_Returns':
                    df[key] = value
            
            # 라벨 분포 확인
            label_dist = df['Label'].value_counts().sort_index()
            label_percentages = (label_dist / len(df) * 100).round(1)
            
            print(f"✅ {symbol_name}: 라벨링 완료")
            print(f"   데이터 포인트: {len(df)}")
            print(f"   라벨 분포: Sell({label_percentages.get(0, 0)}%) Hold({label_percentages.get(1, 0)}%) Buy({label_percentages.get(2, 0)}%)")
            
            return df
            
        except Exception as e:
            print(f"❌ {symbol_name}: 처리 중 오류 - {e}")
            return None

    def process_all_csv_files(self, input_dir, output_dir=None):
        """디렉토리 내 모든 CSV 파일 처리"""
        if output_dir is None:
            output_dir = os.path.join(input_dir, "labeled")
        
        # 출력 디렉토리 생성
        os.makedirs(output_dir, exist_ok=True)
        
        # CSV 파일 목록 가져오기
        csv_files = [f for f in os.listdir(input_dir) if f.endswith('.csv') and 'features_dataset' in f]
        
        if not csv_files:
            print(f"❌ {input_dir}에서 features_dataset CSV 파일을 찾을 수 없습니다.")
            return {}
        
        print(f"🚀 {len(csv_files)}개 CSV 파일 처리 시작...")
        print("="*60)
        
        processed_data = {}
        failed_files = []
        
        for csv_file in tqdm(csv_files, desc="CSV 파일 처리"):
            try:
                # 심볼명 추출 (예: BTC_features_dataset.csv → BTC)
                symbol_name = csv_file.replace('_features_dataset.csv', '')
                
                # 파일 경로
                input_path = os.path.join(input_dir, csv_file)
                
                # 데이터 처리
                processed_df = self.process_crypto_csv(input_path, symbol_name)
                
                if processed_df is not None:
                    # 처리된 데이터 저장
                    processed_data[symbol_name] = processed_df
                    
                    # 라벨링된 CSV 저장
                    output_path = os.path.join(output_dir, f"{symbol_name}_labeled_dataset.csv")
                    processed_df.to_csv(output_path, index=False)
                    
                else:
                    failed_files.append(csv_file)
                    
            except Exception as e:
                print(f"❌ {csv_file}: 파일 처리 중 오류 - {e}")
                failed_files.append(csv_file)
                continue
        
        # 결과 요약
        success_count = len(processed_data)
        total_count = len(csv_files)
        failure_count = total_count - success_count
        
        print(f"\n{'='*60}")
        print(f"🎉 라벨링 작업 완료!")
        print(f"✅ 성공: {success_count}개")
        print(f"❌ 실패: {failure_count}개")
        print(f"📁 출력 경로: {output_dir}")
        
        if failed_files:
            print(f"\n실패한 파일들:")
            for failed in failed_files:
                print(f"  - {failed}")
        
        return processed_data

    def analyze_labels_distribution(self, processed_data):
        """전체 라벨 분포 분석"""
        print(f"\n{'='*60}")
        print("📊 전체 라벨 분포 분석")
        print("="*60)
        
        total_stats = {0: 0, 1: 0, 2: 0}
        total_count = 0
        
        for symbol, df in processed_data.items():
            if 'Label' in df.columns:
                label_counts = df['Label'].value_counts()
                total_count += len(df)
                
                print(f"{symbol:>6}: ", end="")
                for label in [0, 1, 2]:
                    count = label_counts.get(label, 0)
                    percentage = (count / len(df) * 100) if len(df) > 0 else 0
                    total_stats[label] += count
                    
                    label_name = ['Sell', 'Hold', 'Buy'][label]
                    print(f"{label_name} {count:>4}({percentage:>5.1f}%) ", end="")
                print()
        
        print("-"*60)
        print("전체 요약:")
        for label in [0, 1, 2]:
            count = total_stats[label]
            percentage = (count / total_count * 100) if total_count > 0 else 0
            label_name = ['Strong_Sell', 'Hold', 'Strong_Buy'][label]
            print(f"  {label_name}: {count:>6}개 ({percentage:>5.1f}%)")
        
        print(f"  전체: {total_count:>6}개")



In [17]:
if __name__ == "__main__":
    # 라벨러 초기화
    labeler = CryptoLabeler()
    
    # A00_00에서 생성된 CSV 파일들이 있는 디렉토리 경로
    input_directory = "/workspace/AI모델/projects/coin/data/v01/add"
    output_directory = "/workspace/AI모델/projects/coin/data/v01/labeled"
    
    # 모든 CSV 파일 처리
    processed_results = labeler.process_all_csv_files(input_directory, output_directory)
    
    # 라벨 분포 분석
    if processed_results:
        labeler.analyze_labels_distribution(processed_results)
        
        print(f"\n🎯 사용 가능한 심볼: {list(processed_results.keys())}")
        print(f"📈 각 심볼별 라벨링된 CSV는 {output_directory}에 저장되었습니다.")
    else:
        print("❌ 처리된 데이터가 없습니다. 입력 디렉토리와 파일을 확인해주세요.")

🚀 50개 CSV 파일 처리 시작...


CSV 파일 처리:   0%|          | 0/50 [00:00<?, ?it/s]

CSV 파일 처리:   2%|▏         | 1/50 [00:00<00:05,  8.93it/s]

✅ AAVE: 라벨링 완료
   데이터 포인트: 1821
   라벨 분포: Sell(45.0%) Hold(10.0%) Buy(45.0%)
✅ ADA: 라벨링 완료
   데이터 포인트: 2879
   라벨 분포: Sell(45.0%) Hold(10.0%) Buy(45.0%)


CSV 파일 처리:   8%|▊         | 4/50 [00:00<00:05,  8.99it/s]

...

✅ WTRX: 라벨링 완료
   데이터 포인트: 1302
   라벨 분포: Sell(45.0%) Hold(10.0%) Buy(45.0%)
✅ XLM: 라벨링 완료
   데이터 포인트: 2879
   라벨 분포: Sell(45.0%) Hold(10.0%) Buy(45.0%)


CSV 파일 처리:  98%|█████████▊| 49/50 [00:05<00:00,  8.25it/s]

✅ XMR: 라벨링 완료
   데이터 포인트: 2879
   라벨 분포: Sell(45.0%) Hold(10.0%) Buy(45.0%)
✅ XRP: 라벨링 완료
   데이터 포인트: 2879
   라벨 분포: Sell(45.0%) Hold(10.0%) Buy(45.0%)


CSV 파일 처리: 100%|██████████| 50/50 [00:05<00:00,  8.65it/s]


🎉 라벨링 작업 완료!
✅ 성공: 50개
❌ 실패: 0개
📁 출력 경로: /workspace/AI모델/projects/coin/data/v01/labeled

📊 전체 라벨 분포 분석
  AAVE: Sell  820( 45.0%) Hold  182( 10.0%) Buy  819( 45.0%) 
   ADA: Sell 1296( 45.0%) Hold  287( 10.0%) Buy 1296( 45.0%) 
AETHWETH: Sell   62( 44.9%) Hold   14( 10.1%) Buy   62( 44.9%) 
  ALGO: Sell 1031( 45.0%) Hold  228( 10.0%) Buy 1031( 45.0%) 
  AVAX: Sell  825( 45.0%) Hold  183( 10.0%) Buy  825( 45.0%) 
   BCH: Sell 1296( 45.0%) Hold  287( 10.0%) Buy 1296( 45.0%) 
   BGB: Sell  685( 45.0%) Hold  151(  9.9%) Buy  685( 45.0%) 
   BNB: Sell 1296( 45.0%) Hold  287( 10.0%) Buy 1296( 45.0%) 
  BTCB: Sell 1032( 45.0%) Hold  229( 10.0%) Buy 1032( 45.0%) 
   BTC: Sell 1765( 45.0%) Hold  392( 10.0%) Buy 1765( 45.0%) 
CBBTC32994: Sell  171( 45.0%) Hold   38( 10.0%) Buy  171( 45.0%) 
   CRO: Sell 1116( 45.0%) Hold  247( 10.0%) Buy 1116( 45.0%) 
   DAI: Sell  961( 45.0%) Hold  214( 10.0%) Buy  961( 45.0%) 
  DOGE: Sell 1296( 45.0%) Hold  287( 10.0%) Buy 1296( 45.0%) 
   DOT: Sell  839( 45.